###### Imports and Settings

In [110]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
import dataframe_image as dfi
from collections import deque
from functools import reduce
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

In [128]:
data = pd.read_csv('../../data/historicalandprojections.csv')
index = np.array([data.columns.values, data.iloc[0].values])
data.columns = pd.MultiIndex.from_arrays(index)
data = data.iloc[1:]
data.head()

,NAME,GEO_ID,Variable,Decennial,Decennial.1,Decennial.2,Decennial.3,Decennial.4,Decennial.5,Decennial.6,Decennial.7,Decennial.8,Decennial.9,Decennial.10,Decennial.11,Decennial.12,Decennial.13,Decennial.14,Decennial.15,Decennial.16,Decennial.17,Decennial.18,Decennial.19,Decennial.20,Decennial.21,ACS,Decennial.22,ACS.1,WoodsPoole,WoodsPoole.1,WoodsPoole.2,WoodsPoole.3,WoodsPoole.4
,NAME,GEO_ID,Variable,1800.0,1810.0,1820.0,1830.0,1840.0,1850.0,1860.0,1870,1880.0,1890.0,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0,1990.0,2000.0,2010.0,2019.0,2020.0,2020.0,2017.0,2020.0,2025.0,2035.0,2045.0
1,NAME,GEO_ID,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cheatham County,47021,Population,0.0,0.0,0.0,0.0,0.0,0.0,7258.0,6678,7956.0,8845.0,10112.0,10540.0,10039.0,9025.0,9928.0,9167.0,9428.0,13199.0,21616.0,27140.0,35912.0,39105.0,40181.0,41072.0,40539.0,39716.0,41392.0,44245.0,49849.0,54774.0
3,Davidson County,47037,Population,9965.0,15608.0,20154.0,28122.0,30509.0,38882.0,47055.0,62897,79026.0,108174.0,122815.0,149478.0,167815.0,222854.0,257267.0,321758.0,399743.0,448003.0,477811.0,510784.0,569891.0,626681.0,687488.0,715884.0,690540.0,678322.0,699326.0,734235.0,798722.0,848072.0
4,Dickson County,47043,Population,0.0,4516.0,5190.0,7265.0,7074.0,8404.0,9982.0,9340,12460.0,13645.0,18635.0,19955.0,19342.0,18491.0,19718.0,18805.0,18839.0,21977.0,30037.0,35061.0,43156.0,49666.0,52680.0,54315.0,53289.0,51341.0,53116.0,56089.0,61698.0,66221.0
5,Houston County,47083,Population,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4295.0,5390.0,6476.0,6224.0,6212.0,5555.0,6432.0,5318.0,4794.0,5845.0,6871.0,7018.0,8088.0,8426.0,8164.0,8283.0,8201.0,8188.0,8440.0,8862.0,9643.0,10243.0


In [112]:
thing = data.melt(id_vars = [('NAME', 'NAME'), ('GEO_ID', 'GEO_ID'), ('Variable', 'Variable')])
thing.head()

,"(NAME, NAME)","(GEO_ID, GEO_ID)","(Variable, Variable)",variable_0,variable_1,value
0,NAME,GEO_ID,Variable,Decennial,1800.0,NaN
1,Cheatham County,47021,Population,Decennial,1800.0,0.0
2,Davidson County,47037,Population,Decennial,1800.0,9965.0
3,Dickson County,47043,Population,Decennial,1800.0,0.0
4,Houston County,47083,Population,Decennial,1800.0,0.0


In [113]:
thing.columns = ['NAME', 'GEO_ID', 'Variable', 'Source', 'Year', 'Population']

In [114]:
thing.head()

,NAME,GEO_ID,Variable,Source,Year,Population
0,NAME,GEO_ID,Variable,Decennial,1800.0,NaN
1,Cheatham County,47021,Population,Decennial,1800.0,0.0
2,Davidson County,47037,Population,Decennial,1800.0,9965.0
3,Dickson County,47043,Population,Decennial,1800.0,0.0
4,Houston County,47083,Population,Decennial,1800.0,0.0


In [115]:
thing = thing[1:].reset_index(drop = True)
thing.head()

,NAME,GEO_ID,Variable,Source,Year,Population
0,Cheatham County,47021,Population,Decennial,1800.0,0.0
1,Davidson County,47037,Population,Decennial,1800.0,9965.0
2,Dickson County,47043,Population,Decennial,1800.0,0.0
3,Houston County,47083,Population,Decennial,1800.0,0.0
4,Humphreys County,47085,Population,Decennial,1800.0,0.0


In [116]:
thing = thing.drop(columns = 'Variable')
thing.head()

,NAME,GEO_ID,Source,Year,Population
0,Cheatham County,47021,Decennial,1800.0,0.0
1,Davidson County,47037,Decennial,1800.0,9965.0
2,Dickson County,47043,Decennial,1800.0,0.0
3,Houston County,47083,Decennial,1800.0,0.0
4,Humphreys County,47085,Decennial,1800.0,0.0


In [117]:
thing = thing.replace('nan', '0')
thing = thing.replace(' ', '0')

In [118]:
numeros = ['Year', 'Population']
thing[numeros] = thing[numeros].astype(float)

In [129]:
thing['Year'] = thing['Year'].astype(int)

In [130]:
thing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2519 entries, 0 to 2518
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NAME        2519 non-null   object 
 1   GEO_ID      2159 non-null   object 
 2   Source      2519 non-null   object 
 3   Year        2519 non-null   int32  
 4   Population  1092 non-null   float64
dtypes: float64(1), int32(1), object(3)
memory usage: 88.7+ KB


In [131]:
thing.tail()

,NAME,GEO_ID,Source,Year,Population
2514,White House city,80200,WoodsPoole,2045,NaN
2515,White House [Robertson],NaN,WoodsPoole,2045,NaN
2516,White House [Sumner],NaN,WoodsPoole,2045,NaN
2517,Tennessee,0400000US47,WoodsPoole,2045,8716671.0
2518,USA,0100000US,WoodsPoole,2045,NaN


In [124]:
takeoffsuffix = thing['Source'].str.split(pat = '.', expand = True)

In [125]:
thing['Source'] = takeoffsuffix[0]

In [127]:
thing.to_csv('../../data/FormattedPopulationTimeSeries.csv', index = False)

In [139]:
def percentchange(x, y):
    return (x-y)*100/y
def realchange(x, y):
    return x-y

In [155]:
data = pd.read_csv('../../data/HistoricalPopulation.csv')
data = data.set_index(['NAME', 'GEO_ID'])
data.columns = pd.MultiIndex.from_product([['Population'], data.columns, ['None']])
data.head()

Population                                                                                         \
                                                    1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900   1910   1920   1930    1940    1950   
                                                    None None None None None None None None None None None   None   None   None    None    None   
NAME                         GEO_ID                                                                                                               
Adams city, Tennessee        1600000US4700200        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  542.0  672.0    NaN     NaN     NaN   
Ashland City town, Tennessee 1600000US4702180        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  641.0  649.0  712.0   957.0  1024.0   
Belle Meade city, Tennessee  1600000US4704620        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN    NaN  2061.0  2831.0   
Berry Hill city, Tennessee   1600000US4705140        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN    NaN     NaN  1248.0   
Brentwood city, Tennessee    1600000US4708280        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN    NaN     NaN     NaN   

                                                                                                     
                                                 1960    1970    1980     1990   2000   2010   2020  
                                                 None    None    None     None   None   None   None  
NAME                         GEO_ID                                                                  
Adams city, Tennessee        1600000US4700200     NaN   458.0   600.0    587.0    566    633    624  
Ashland City town, Tennessee 1600000US4702180  1400.0  2027.0  2329.0   2552.0   3641   4541   5193  
Belle Meade city, Tennessee  1600000US4704620  3082.0     NaN  3182.0   2839.0   2943   2912   2901  
Berry Hill city, Tennessee   1600000US4705140  1551.0     NaN  1113.0    802.0    674    537   2112  
Brentwood city, Tennessee    1600000US4708280     NaN     NaN  9431.0  16392.0  23445  37060  45373

In [156]:
data[('Population % Change', 'None','1800-1810')] = percentchange(data[('Population','1810','None')], data[('Population','1800','None')])
data[('Population % Change', 'None','1810-1820')] = percentchange(data[('Population','1820','None')], data[('Population','1810','None')])
data[('Population % Change', 'None','1820-1830')] = percentchange(data[('Population','1830','None')], data[('Population','1820','None')])
data[('Population % Change', 'None','1830-1840')] = percentchange(data[('Population','1840','None')], data[('Population','1830','None')])
data[('Population % Change', 'None','1840-1850')] = percentchange(data[('Population','1850','None')], data[('Population','1840','None')])
data[('Population % Change', 'None','1850-1860')] = percentchange(data[('Population','1860','None')], data[('Population','1850','None')])
data[('Population % Change', 'None','1860-1870')] = percentchange(data[('Population','1870','None')], data[('Population','1860','None')])
data[('Population % Change', 'None','1870-1880')] = percentchange(data[('Population','1880','None')], data[('Population','1870','None')])
data[('Population % Change', 'None','1880-1890')] = percentchange(data[('Population','1890','None')], data[('Population','1880','None')])
data[('Population % Change', 'None','1890-1900')] = percentchange(data[('Population','1900','None')], data[('Population','1890','None')])
data[('Population % Change', 'None','1900-1910')] = percentchange(data[('Population','1910','None')], data[('Population','1900','None')])
data[('Population % Change', 'None','1910-1920')] = percentchange(data[('Population','1920','None')], data[('Population','1910','None')])
data[('Population % Change', 'None','1920-1930')] = percentchange(data[('Population','1930','None')], data[('Population','1920','None')])
data[('Population % Change', 'None','1930-1940')] = percentchange(data[('Population','1940','None')], data[('Population','1930','None')])
data[('Population % Change', 'None','1940-1950')] = percentchange(data[('Population','1950','None')], data[('Population','1940','None')])
data[('Population % Change', 'None','1950-1960')] = percentchange(data[('Population','1960','None')], data[('Population','1950','None')])
data[('Population % Change', 'None','1960-1970')] = percentchange(data[('Population','1970','None')], data[('Population','1960','None')])
data[('Population % Change', 'None','1970-1980')] = percentchange(data[('Population','1980','None')], data[('Population','1970','None')])
data[('Population % Change', 'None','1980-1990')] = percentchange(data[('Population','1990','None')], data[('Population','1980','None')])
data[('Population % Change', 'None','1990-2000')] = percentchange(data[('Population','2000','None')], data[('Population','1990','None')])
data[('Population % Change', 'None','2000-2010')] = percentchange(data[('Population','2010','None')], data[('Population','2000','None')])
data[('Population % Change', 'None','2010-2020')] = percentchange(data[('Population','2020','None')], data[('Population','2010','None')])

data[('Population Change', 'None','1800-1810')] = realchange(data[('Population','1810','None')], data[('Population','1800','None')])
data[('Population Change', 'None','1810-1820')] = realchange(data[('Population','1820','None')], data[('Population','1810','None')])
data[('Population Change', 'None','1820-1830')] = realchange(data[('Population','1830','None')], data[('Population','1820','None')])
data[('Population Change', 'None','1830-1840')] = realchange(data[('Population','1840','None')], data[('Population','1830','None')])
data[('Population Change', 'None','1840-1850')] = realchange(data[('Population','1850','None')], data[('Population','1840','None')])
data[('Population Change', 'None','1850-1860')] = realchange(data[('Population','1860','None')], data[('Population','1850','None')])
data[('Population Change', 'None','1860-1870')] = realchange(data[('Population','1870','None')], data[('Population','1860','None')])
data[('Population Change', 'None','1870-1880')] = realchange(data[('Population','1880','None')], data[('Population','1870','None')])
data[('Population Change', 'None','1880-1890')] = realchange(data[('Population','1890','None')], data[('Population','1880','None')])
data[('Population Change', 'None','1890-1900')] = realchange(data[('Population','1900','None')], data[('Population','1890','None')])
data[('Population Change', 'None','1900-1910')] = realchange(data[('Population','1910','None')], data[('Population','1900','None')])
data[('Population Change', 'None','1910-1920')] = realchange(data[('Population','1920','None')], data[('Population','1910','None')])
data[('Population Change', 'None','1920-1930')] = realchange(data[('Population','1930','None')], data[('Population','1920','None')])
data[('Population Change', 'None','1930-1940')] = realchange(data[('Population','1940','None')], data[('Population','1930','None')])
data[('Population Change', 'None','1940-1950')] = realchange(data[('Population','1950','None')], data[('Population','1940','None')])
data[('Population Change', 'None','1950-1960')] = realchange(data[('Population','1960','None')], data[('Population','1950','None')])
data[('Population Change', 'None','1960-1970')] = realchange(data[('Population','1970','None')], data[('Population','1960','None')])
data[('Population Change', 'None','1970-1980')] = realchange(data[('Population','1980','None')], data[('Population','1970','None')])
data[('Population Change', 'None','1980-1990')] = realchange(data[('Population','1990','None')], data[('Population','1980','None')])
data[('Population Change', 'None','1990-2000')] = realchange(data[('Population','2000','None')], data[('Population','1990','None')])
data[('Population Change', 'None','2000-2010')] = realchange(data[('Population','2010','None')], data[('Population','2000','None')])
data[('Population Change', 'None','2010-2020')] = realchange(data[('Population','2020','None')], data[('Population','2010','None')])

In [157]:
data.tail()

Population                                                                                            \
                                                    1800      1810      1820      1830      1840      1850      1860      1870      1880      1890   
                                                    None      None      None      None      None      None      None      None      None      None   
NAME                         GEO_ID                                                                                                                  
White House city, Tennessee  1600000US4780200        NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
Williamson County, Tennessee 0500000US47187       2868.0   13153.0   20640.0   26638.0   27006.0   27201.0   23827.0   25328.0   28313.0   26321.0   
Wilson County, Tennessee     0500000US47189       3261.0   11952.0   18730.0   25472.0   24460.0   27443.0   26072.0   25881.0   28747.0   27148.0   
MPO                          NaN                 24990.0   82399.0  130366.0  167872.0  170687.0  191030.0  194665.0  223561.0  255217.0  278598.0   
GNRC                         NaN                 27889.0  100709.0  160239.0  202641.0  208470.0  236620.0  251792.0  285671.0  339124.0  365938.0   

                                                                                                                                                     \
                                                   1900      1910      1920      1930      1940      1950      1960      1970       1980       1990   
                                                   None      None      None      None      None      None      None      None       None       None   
NAME                         GEO_ID                                                                                                                   
White House city, Tennessee  1600000US4780200       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN     2225.0     2987.0   
Williamson County, Tennessee 0500000US47187     26429.0   24213.0   23409.0   22845.0   25220.0   24307.0   25267.0   34330.0    58108.0    81021.0   
Wilson County, Tennessee     0500000US47189     27078.0   25394.0   26241.0   23929.0   25267.0   26318.0   27668.0   36999.0    56064.0    67675.0   
MPO                          NaN               303669.0  323827.0  339256.0  392743.0  443480.0  514004.0  610297.0  707342.0   849947.0   977637.0   
GNRC                         NaN               409535.0  428860.0  441256.0  487642.0  544987.0  617205.0  723279.0  837118.0  1022572.0  1178548.0   

                                                                         Population % Change                                                        \
                                                  2000     2010     2020                None                                                         
                                                  None     None     None           1800-1810  1810-1820  1820-1830 1830-1840  1840-1850  1850-1860   
NAME                         GEO_ID                                                                                                                  
White House city, Tennessee  1600000US4780200     7220    10255    12982                 NaN        NaN        NaN       NaN        NaN        NaN   
Williamson County, Tennessee 0500000US47187     126638   183182   247726          358.612273  56.922375  29.060078  1.381485   0.722062 -12.403956   
Wilson County, Tennessee     0500000US47189      88809   113993   147737          266.513339  56.710174  35.995729 -3.972990  12.195421  -4.995809   
MPO                          NaN               1221741  1494344  1822891          229.727891  58.213085  28.769771  1.676873  11.918307   1.902842   
GNRC                         NaN               1481223  1803604  2190892          261.106529  59.110904  26.461723  2.876516  13.503142

In [158]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_2':'Year', 'level_3':'Time Frame'})
data.head()

,NAME,GEO_ID,Year,Time Frame,Population,Population % Change,Population Change
0,"Adams city, Tennessee",1600000US4700200,1910,None,542.0,NaN,NaN
1,"Adams city, Tennessee",1600000US4700200,1920,None,672.0,NaN,NaN
2,"Adams city, Tennessee",1600000US4700200,1970,None,458.0,NaN,NaN
3,"Adams city, Tennessee",1600000US4700200,1980,None,600.0,NaN,NaN
4,"Adams city, Tennessee",1600000US4700200,1990,None,587.0,NaN,NaN
